In [34]:
import pyDOE2 as ff
import numpy as np
import pandas as pd
from scipy.stats import f as Fisher
from scipy.stats import t as Student

In [35]:
Authors = ["Danis","Myrat","Alia","Misha"]
Author = Authors[3]
Author

'Misha'

In [49]:
factor_count = 2  # Количество факторов
y_count = 3  # Количество опытов для каждой комбинации факторов
N0 = factor_count**2
experiment_count = N0 + 2 * factor_count + 1  # Количество экспериментов
name_y = 'y average'  # Название столбца для построения регрессии
alpha = int(np.sqrt((np.sqrt(N0 * experiment_count) - N0)/2))
betta = (N0 + 2*alpha**2)/experiment_count
print(f"α = {alpha} ; β = {betta}")
table = ff.ccdesign(factor_count, center=(0, 1))  # Матрица планирования
table = np.array(table, dtype=np.int8)
table


α = 1 ; β = 0.6666666666666666


array([[-1, -1],
       [ 1, -1],
       [-1,  1],
       [ 1,  1],
       [-1,  0],
       [ 1,  0],
       [ 0, -1],
       [ 0,  1],
       [ 0,  0]], dtype=int8)

In [37]:
database = pd.read_excel('Data_' + Author + '.xlsx') 
database = database[database.columns[1:]]
database = database.to_numpy()
#print(database)
data = []
for i in range(y_count):
    data.append(list(zip(np.log(database[y_count*i,:] - 0.43),
                         np.log(database[y_count*i + 1,:]- 0.43),
                         np.log(database[y_count*i + 2,:]- 0.43)))) # хранение опытов для каждой комбинации факторов
    # data.append(list(zip(database[y_count*i,:],
    #                      database[y_count*i + 1,:],
    #                      database[y_count*i + 2,:]))) # хранение опытов для каждой комбинации факторов
# data


In [38]:
dtypes = np.dtype(
    [   
        ("x0", int),
        ("x1", int),
        ("x2", int),
        ("x1x2", int),
        ("(x1)^2", int),
        ("(x2)^2", int),
        ("(x1~)^2", np.float64),
        ("(x2~)^2", np.float64),
        ("y1", np.float64),
        ("y2", np.float64),
        ('y3', np.float64),
        ('y average', np.float64)
    ]
)
df = pd.DataFrame(np.zeros(experiment_count, dtype=dtypes))

In [39]:
for i in range(experiment_count):
    idx1 = table[i, 0]  # x1
    idx2 = table[i, 1]  # x2
    # совокупность значений y
    ys = np.array(data[idx1 + 1][idx2 + 1], dtype=np.float64)
    # print(ys)
    df.iloc[i] = {'x0': 1, 'x1': idx1, 'x2': idx2, 'x1x2': idx1*idx2, '(x1)^2': idx1**2, '(x2)^2': idx2**2,
                '(x1~)^2': idx1**2 - betta, '(x2~)^2': idx2**2 - betta,
                'y1': ys[0], 'y2': ys[1], 'y3': ys[2], 'y average': np.mean(ys)}
# y1 - столбец целевой переменной для построения регрессии
fixed_y = df[name_y]
# df


In [40]:

a = []
# Линейные коэффициенты
tmp = np.zeros(factor_count + 1)
for i,name in enumerate(df.columns[:factor_count + 1]):
    tmp[i] = (1./(N0 + 2*alpha**2)) * np.sum(df[name] * fixed_y)
tmp[0] = (1./experiment_count) * np.sum(fixed_y)
a.append(tmp)
del tmp
# Квадратичные коэффициенты
tmp = np.zeros(factor_count)
for i,name in enumerate(df.columns[(factor_count + 1)*2:(factor_count + 1)*2 + factor_count]):
    tmp[i] = (1./(2*alpha**4)) * np.sum(df[name] * fixed_y)
a.append(tmp)
del tmp
# Парный коэффициент
a.append((1./N0) * np.sum(df['x1x2'] * fixed_y))
print(f"Линейные коэффициенты: {a[0]}\nКвадратичные коэффициенты: {a[1]}\nПарный коэффициент: {a[2]}")
regression = lambda x: np.sum(x*a[0]) + np.sum((x[1:]**2)*a[1]) + x[1]*x[2]*a[2]   #где x[0]=1 - фиктивный фактор

Линейные коэффициенты: [-0.64046043  0.37458774  0.20467431]
Квадратичные коэффициенты: [-0.19216315 -0.12795978]
Парный коэффициент: 0.007605610616588376


In [41]:
dtypes = np.dtype(
    [
        ('(y1-y_avg)^2', np.float64),
        ('(y2-y_avg)^2', np.float64),
        ('(y3-y_avg)^2', np.float64),
        ("Variance", np.float64),
        ('y regression_' + name_y, np.float64),
        ('Δy^2', np.float64)
    ]
)
df_tmp = pd.DataFrame(np.zeros(experiment_count, dtype=dtypes))
df = pd.concat([df, df_tmp], axis=1)
# df

In [42]:
# df['y regression_' + name_y] = [regression(i) for i in df[df.columns[:factor_count + 1]].to_numpy()]
# df['Δy^2'] = (fixed_y - df['y regression_' + name_y]) ** 2

tmp = df.columns[factor_count*3 + 2:(factor_count*3 + 2) + y_count]

for idx,name in enumerate(df_tmp.columns[:y_count]):
    df[name] = (df[tmp[idx]] - df['y average'])**2
    df['Variance'] += df[name]
del tmp

# df

### Проверка адекватности модели ###

In [43]:
f = experiment_count - (factor_count + 1)
print(f"f = {f}")

S_y = df['Variance'].sum() * (1/(experiment_count * (y_count - 1)))
print(f"S_y = {S_y}")


f = 6
S_y = 0.0012561908155534916


In [44]:
p = 0.01 # Принятый уровень значимости

#### Критерий Стьюдента ####

In [45]:
S_a0 = np.sqrt(np.abs(S_y/(y_count*(experiment_count+2*(alpha**4)))))
S_ai = np.sqrt(np.abs(S_y/(y_count*(N0+2*(alpha**2)))))
S_ail = np.sqrt(np.abs(S_y/(y_count*N0)))
S_aii = np.sqrt(np.abs(S_y/(y_count*(2*(alpha**4)))))
print(f"S_a0 = {S_a0}\nS_ai = {S_ai}\nS_ail = {S_ail}\nS_aii = {S_aii}")

t_table = Student.ppf(1-p/2, experiment_count*(y_count-1)) # Табличный критерий Стьюдента
print(f"Табличное значение критерия Стьюдента: {t_table}")

t = np.zeros(6)
# Проверка линейных коэффициентов
t[0] = np.abs(a[0][0])/S_a0
for i in range(1, len(a[0])):
    t[i] = np.abs(a[0][i])/S_ai
    if t[i] > t_table:
        print(f"Коэффициент a{i} является значимым!")
    else:
        print(f"Коэффициент a{i} является незначимым!")
        a[0][i] = 0
# Проверка квадратичных коэффициентов
for i in range(len(a[1])):
    t[i+len(a[0])] = np.abs(a[1][i])/S_aii
    if t[i+len(a[0])] > t_table:
        print(f"Коэффициент a{i+1}{i+1} является значимым!")
    else:
        print(f"Коэффициент a{i+1}{i+1} является незначимым!")
        a[1][i] = 0
# Проверка парного коэффициента:
t[-1] = np.abs(a[-1])/S_ail
if t[-1] > t_table:
    print(f"Коэффициент a12 является значимым!")
else:
    print(f"Коэффициент a12 является незначимым!")
    a[-1] = 0

S_a0 = 0.006169796459374151
S_ai = 0.008353943897457096
S_ail = 0.01023144994430364
S_aii = 0.014469455273975657
Табличное значение критерия Стьюдента: 2.878440472713585
Коэффициент a1 является значимым!
Коэффициент a2 является значимым!
Коэффициент a11 является значимым!
Коэффициент a22 является значимым!
Коэффициент a12 является незначимым!


#### Критерий Фишера ####

In [46]:
df['y regression_' + name_y] = [regression(i) for i in df[df.columns[:factor_count + 1]].to_numpy()]
df['Δy^2'] = (fixed_y - df['y regression_' + name_y]) ** 2

S_ad = (1./f) * df['Δy^2'].sum()
print(f"S_ад = {S_ad}")

S_ад = 0.06842919051345227


In [47]:
F_table = Fisher.ppf(1-p, f, experiment_count*(y_count-1)) # Табличный критерий Фишера
print(f"Табличное значение критерия Фишера: {F_table}")

F = S_ad/S_y
print(f"Модель адекватна c F = {F}!") if F < F_table else print(f"Модель неадекватна c F = {F}!")


Табличное значение критерия Фишера: 4.014636507354758
Модель неадекватна c F = 54.47356378202909!


In [48]:
df

,x0,x1,x2,x1x2,(x1)^2,(x2)^2,(x1~)^2,(x2~)^2,y1,y2,y3,y average,(y1-y_avg)^2,(y2-y_avg)^2,(y3-y_avg)^2,Variance,y regression_y average,Δy^2
0,1,-1,-1,1,1,1,0.333333,0.333333,-1.319010,-1.350155,-1.299750,-1.322972,0.000016,7.389564e-04,0.000539,0.001294,-1.539845,0.047034
1,1,1,-1,-1,1,1,0.333333,0.333333,-0.597110,-0.620827,-0.548008,-0.588648,0.000072,1.035437e-03,0.001652,0.002759,-0.790670,0.040813
2,1,-1,1,-1,1,1,0.333333,0.333333,-0.947781,-0.949589,-0.856608,-0.917993,0.000887,9.983263e-04,0.003768,0.005654,-1.130497,0.045158
3,1,1,1,1,1,1,0.333333,0.333333,-0.138228,-0.159582,-0.161931,-0.153247,0.000226,4.013287e-05,0.000075,0.000341,-0.381321,0.052018
4,1,-1,0,0,1,0,0.333333,-0.666667,-1.037894,-0.931151,-1.019986,-0.996343,0.001726,4.250098e-03,0.000559,0.006535,-1.207211,0.044465
5,1,1,0,0,1,0,0.333333,-0.666667,-0.230042,-0.247564,-0.266051,-0.247886,0.000318,1.034038e-07,0.000330,0.000648,-0.458036,0.044163
6,1,0,-1,0,0,1,-0.666667,0.333333,-0.784167,-0.735637,-0.735429,-0.751744,0.001051,2.594321e-04,0.000266,0.001577,-0.973095,0.048996
7,1,0,1,0,0,1,-0.666667,0.333333,-0.361113,-0.339116,-0.392006,-0.364078,0.000009,6.231345e-04,0.000780,0.001412,-0.563746,0.039867
8,1,0,0,0,0,0,-0.666667,-0.666667,-0.383606,-0.451614,-0.428478,-0.421232,0.001416,9.230228e-04,0.000052,0.002391,-0.640460,0.048061


### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Мурат&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Данис ###
## &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;↓&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ↓ ##

![cat](https://www.meme-arsenal.com/memes/27366f67c1b492100085bfebf3415ca2.jpg)

### -Как найти c,m,n? ###
### -Что за t? ###
### -Ты кто? ###